In [ ]:
D:\WM\Project\ladder\chromedriver_win32

In [2]:
import sys

In [3]:
sys.path

['D:\\WM\\Project\\ladder',
 'D:\\WM\\Project\\ladder\\PYTHONPATH=\\path\\to\\dlib\\python_examples:$PYTHONPATH',
 'C:\\Users\\54308\\AppData\\Local\\Programs\\Python\\Python37\\python37.zip',
 'C:\\Users\\54308\\AppData\\Local\\Programs\\Python\\Python37\\DLLs',
 'C:\\Users\\54308\\AppData\\Local\\Programs\\Python\\Python37\\lib',
 'C:\\Users\\54308\\AppData\\Local\\Programs\\Python\\Python37',
 'd:\\wm\\project\\quora',
 '',
 'd:\\wm\\project\\quora\\lib\\site-packages',
 'd:\\wm\\project\\quora\\lib\\site-packages\\win32',
 'd:\\wm\\project\\quora\\lib\\site-packages\\win32\\lib',
 'd:\\wm\\project\\quora\\lib\\site-packages\\Pythonwin',
 'd:\\wm\\project\\quora\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\54308\\.ipython']

In [1]:
import selenium
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
from selenium import webdriver

DRIVER_PATH = "D:\WM\Project\ladder\chromedriver_win32\chromedriver.exe"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [2]:
wd.get('https://google.com')

In [50]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    #search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    search_url = "https://www.pexels.com/search/{q}"
    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            #return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [51]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [52]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [53]:
import os
import time
import requests
import io
from PIL import Image
import hashlib

In [54]:
search_term = "ladder"
search_and_download(search_term,DRIVER_PATH,target_path='./images',number_images=10)

Found: 0 search results. Extracting links from 0:0
Found: 0 image links, looking for more ...


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".mye4qd"}
  (Session info: chrome=87.0.4280.88)


In [7]:
search_term = "ladders yard"
search_and_download(search_term,DRIVER_PATH,target_path='./images',number_images=150)

Found: 100 search results. Extracting links from 0:100
Found: 151 image links, done!
SUCCESS - saved https://www.momendeavors.com/wp-content/uploads/2012/06/DIY-Patriotic-Ladder-Golf-Game.jpg - as ./images\ladders_yard\ce1ef8bef7.jpg
SUCCESS - saved https://cdn.statically.io/img/www.loveandrenovations.com/wp-content/uploads/2017/04/TIERED-LADDER-GARDEN-1.png?quality=100&f=auto - as ./images\ladders_yard\17b6022455.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT9s8iMRkLW7QBbPfWDnC4Pfk5bfCR3kgT3Ig&usqp=CAU - as ./images\ladders_yard\9d9d025575.jpg
SUCCESS - saved https://www.wikihow.com/images/3/38/Use-an-Extension-Ladder-Step-2-Version-5.jpg - as ./images\ladders_yard\a169ce1251.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSZelekkcEn_138goOkC8HrYyY0kJ6v8Wt70w&usqp=CAU - as ./images\ladders_yard\f7bc32c33f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSqYlviVCwCGhVdLofG4ARu_OpuDU0m4b2-KA&usqp=CAU - as ./

SUCCESS - saved https://1.bp.blogspot.com/-BZPA7SWMvjM/Wo6ycn7gQmI/AAAAAAAAyQQ/WtNRC15zSYUcUIlP6D-iGm-HUfDqbXC1QCLcBGAs/s1600/IMG_5044.JPG - as ./images\ladders_yard\2c8628a383.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRE3NsU3Wp5sXevZhL03nNuBgGR5xZCCUsWrw&usqp=CAU - as ./images\ladders_yard\08f25f1098.jpg
SUCCESS - saved https://www.houseofhawthornes.com/wp-content/uploads/2016/03/Ladders-In-The-Garden.jpg - as ./images\ladders_yard\5dbb7120cb.jpg
ERROR - Could not save https://www.fleamarketgardening.org/wp-content/uploads/2015/04/Annie-Downss-rustic-garden-ladder.jpg?e768a3 - cannot identify image file <_io.BytesIO object at 0x000002C5248B9F48>
SUCCESS - saved https://cdn.thewirecutter.com/wp-content/uploads/2017/08/ladders-0083-little-giant-select-step-630.jpg - as ./images\ladders_yard\28b7213590.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRdIZsBVW4852TouhBbvse0UUQJd1tjYk2fpw&usqp=CAU - as ./images\ladders_yard\ac48ed0cc

SUCCESS - saved https://cdn-fastly.hometalk.com/media/aa1jct1r8nds833oun2951d0d6661e5ef.jpg?size=720x845&nocrop=1 - as ./images\ladders_yard\fe60eb4fd4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ0k_O7IxX6viJ33tL9krBHigPc-mNADSYKJg&usqp=CAU - as ./images\ladders_yard\db387d3b13.jpg
SUCCESS - saved https://cdn.thewirecutter.com/wp-content/uploads/2017/08/ladders-0085-gorilla-glf-5x-fiberglass-hybrid-630.jpg - as ./images\ladders_yard\5afbc420fa.jpg
SUCCESS - saved https://i.ytimg.com/vi/jaDEO0WUPPU/maxresdefault.jpg - as ./images\ladders_yard\52dc502dd6.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTNf5JLD7GviGm2nk6dASTd3cgWmpRpR6JMLg&usqp=CAU - as ./images\ladders_yard\8b19621511.jpg
SUCCESS - saved https://images-na.ssl-images-amazon.com/images/I/51vZCKuHLML._AC_SX466_.jpg - as ./images\ladders_yard\10489cef0c.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/f/f0/Ladder_Golf%2C_Homerville.JPG - as ./images\la

## rename

In [26]:
path = "yolov4/darknet/images/ladders_yard/"
for count, filename in enumerate(os.listdir(path)): 
    src = path+filename
    dst = str(count)+".jpg"
    dst = path+dst
    os.rename(src,dst)

In [ ]:
dst ="Hostel" + str(count) + ".jpg"
        src ='xyz'+ filename 
        dst ='xyz'+ dst 
          
        # rename() function will 
        # rename all the files 
        os.rename(src, dst) 

In [27]:
os.listdir("yolov4/darknet/images/ladders_yard/")

['0.jpg',
 '0.txt',
 '1.jpg',
 '1.txt',
 '10.jpg',
 '10.txt',
 '100.jpg',
 '101.jpg',
 '102.jpg',
 '102.txt',
 '103.jpg',
 '104.jpg',
 '104.txt',
 '105.jpg',
 '106.jpg',
 '106.txt',
 '107.jpg',
 '107.txt',
 '108.jpg',
 '108.txt',
 '109.jpg',
 '109.txt',
 '11.jpg',
 '11.txt',
 '110.jpg',
 '110.txt',
 '111.jpg',
 '111.txt',
 '112.jpg',
 '112.txt',
 '113.jpg',
 '113.txt',
 '114.jpg',
 '114.txt',
 '115.jpg',
 '115.txt',
 '116.jpg',
 '116.txt',
 '117.jpg',
 '117.txt',
 '118.jpg',
 '118.txt',
 '119.jpg',
 '12.jpg',
 '12.txt',
 '120.jpg',
 '120.txt',
 '121.jpg',
 '121.txt',
 '122.jpg',
 '122.txt',
 '123.jpg',
 '123.txt',
 '124.jpg',
 '124.txt',
 '125.jpg',
 '125.txt',
 '126.jpg',
 '126.txt',
 '127.jpg',
 '128.jpg',
 '128.txt',
 '129.jpg',
 '129.txt',
 '13.jpg',
 '13.txt',
 '130.jpg',
 '130.txt',
 '131.jpg',
 '131.txt',
 '132.jpg',
 '132.txt',
 '133.jpg',
 '133.txt',
 '134.jpg',
 '134.txt',
 '135.jpg',
 '135.txt',
 '136.jpg',
 '136.txt',
 '137.jpg',
 '137.txt',
 '138.jpg',
 '138.txt',
 '139.jp

In [1]:
from datetime import datetime

now = datetime.now()

In [2]:
now

datetime.datetime(2021, 1, 11, 10, 55, 42, 836652)

In [8]:
str(now.month)

'1'

In [7]:
str(now.year)+str(now.month)+str(now.day)+str(now.hour)

'202111110'

In [11]:
now.strftime("%Y-%m-%d_%H:%M:%S")

'2021-01-11_10:55:42'

In [ ]:
"%Y-%m-%d %H:%M:%S"

In [61]:
import os

In [87]:
cur_files = os.listdir("yolov4/darknet/images/ladders/ladder")

In [88]:
len(cur_files)

141

In [70]:
path = "yolov4/darknet/images/ladders/ladder/"

In [86]:
for name in os.listdir("yolov4/darknet/images/ladders/ladder"):
    pre,post = name.split(".")
    if post=="jpg":
        if pre+".txt" not in cur_files:
            print(name)
            os.remove(path+name)

In [89]:
jpg_files = [f for f in cur_files if f.endswith(".jpg")]

In [91]:
len(jpg_files)

70

In [92]:
r_path="images/ladders/ladder/"
tr = 65
te = 5
tr_files = []
te_files = []

In [93]:
for i,f in enumerate(jpg_files):
    if i<tr:
        tr_files.append(r_path+f)
    else:
        te_files.append(r_path+f)

In [94]:
len(tr_files)

65

In [95]:
with open('yolov4/darknet/images/ladder_training_0106.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(tr_files))

In [96]:
with open('yolov4/darknet/images/ladder_testing_0106.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(te_files))

In [48]:
import cv2

In [49]:
cv2.__version__

'4.5.1'

# detect

In [3]:
import numpy as  np
import cv2

# load the image to detect, get width, height 

In [12]:
from datetime import datetime
import os
now = datetime.now()

In [17]:
# Source file path 
src = 'D:\WM\Project\ladder\yolov4\custom_tools\apps\Yolo4-ladder-detection-flask-app\backend\model_config\ladder\images'
  
# Destination file path 
cur = 'D:\WM\Project\ladder\yolov4\darknet\images\\'
dst =cur+"ladder_"+now.strftime("%Y-%m-%d_%H_%M")
if not os.path.exists(dst):
    os.mkdir(dst)
# Create a symbolic link 
# pointing to src named dst 
# using os.symlink() method 
os.symlink(src, dst) 

OSError: symbolic link privilege not held

In [38]:
img_to_detect2 = cv2.imread('yolov4/darknet/images/ladders/ladder/094.jpg')

In [40]:
img_to_detect2

array([[[ 61,  72,  80],
        [ 47,  57,  67],
        [ 33,  40,  59],
        ...,
        [  6,  18,   0],
        [  9,  21,   3],
        [ 11,  23,   5]],

       [[ 56,  68,  74],
        [ 43,  54,  62],
        [ 30,  38,  55],
        ...,
        [  6,  18,   0],
        [ 10,  22,   4],
        [ 12,  24,   6]],

       [[ 45,  58,  60],
        [ 34,  46,  50],
        [ 24,  33,  46],
        ...,
        [  9,  20,   4],
        [ 12,  23,   7],
        [ 15,  26,  10]],

       ...,

       [[ 92, 108, 115],
        [ 53,  69,  76],
        [ 50,  65,  74],
        ...,
        [ 87, 100, 102],
        [ 78,  90,  94],
        [ 33,  45,  51]],

       [[101, 117, 124],
        [ 71,  87,  94],
        [ 44,  60,  67],
        ...,
        [ 92, 105, 107],
        [ 90, 102, 106],
        [ 45,  57,  63]],

       [[ 95, 111, 118],
        [ 83,  99, 106],
        [ 51,  67,  74],
        ...,
        [ 99, 112, 114],
        [115, 127, 131],
        [ 90, 102, 108]]

In [57]:
img_to_detect = cv2.imread('yolov4/darknet/images/ladders/ladder/026.jpg')

In [58]:
img_height = img_to_detect.shape[0]
img_width = img_to_detect.shape[1]

# convert to blob to pass into model
img_blob = cv2.dnn.blobFromImage(img_to_detect, 0.003922, (416, 416), swapRB=True, crop=False)
#recommended by yolo authors, scale factor is 0.003922=1/255, width,height of blob is 320,320
#accepted sizes are 320×320,416×416,608×608. More size means more accuracy but less speed

# set of 80 class labels 
class_labels = ["ladder"]

#Declare List of colors as an array
#Green, Blue, Red, cyan, yellow, purple
#Split based on ',' and for every split, change type to int
#convert that to a numpy array to apply color mask to the image numpy array
class_colors = ["0,255,0","0,0,255","255,0,0","255,255,0","0,255,255"]
class_colors = [np.array(every_color.split(",")).astype("int") for every_color in class_colors]
class_colors = np.array(class_colors)
class_colors = np.tile(class_colors,(16,1))

# Loading pretrained model 
# input preprocessed blob into model and pass through the model
# obtain the detection predictions by the model using forward() method
yolo_model = cv2.dnn.readNetFromDarknet('yolov4/darknet/ladder_yolov4.cfg','yolov4/darknet/ladder_yolov4_best.weights')

# Get all layers from the yolo network
# Loop and find the last layer (output layer) of the yolo network 
yolo_layers = yolo_model.getLayerNames()
yolo_output_layer = [yolo_layers[yolo_layer[0] - 1] for yolo_layer in yolo_model.getUnconnectedOutLayers()]

# input preprocessed blob into model and pass through the model
yolo_model.setInput(img_blob)
# obtain the detection layers by forwarding through till the output layer
obj_detection_layers = yolo_model.forward(yolo_output_layer)


############## NMS Change 1 ###############
# initialization for non-max suppression (NMS)
# declare list for [class id], [box center, width & height[], [confidences]
class_ids_list = []
boxes_list = []
confidences_list = []
############## NMS Change 1 END ###########


# loop over each of the layer outputs
for object_detection_layer in obj_detection_layers:
    # loop over the detections
    for object_detection in object_detection_layer:
        
        # obj_detections[1 to 4] => will have the two center points, box width and box height
        # obj_detections[5] => will have scores for all objects within bounding box
        all_scores = object_detection[5:]
        predicted_class_id = np.argmax(all_scores)
        prediction_confidence = all_scores[predicted_class_id]
    
        # take only predictions with confidence more than 50%
        if prediction_confidence > 0.50:

            #obtain the bounding box co-oridnates for actual image from resized image size
            bounding_box = object_detection[0:4] * np.array([img_width, img_height, img_width, img_height])
            (box_center_x_pt, box_center_y_pt, box_width, box_height) = bounding_box.astype("int")
            start_x_pt = int(box_center_x_pt - (box_width / 2))
            start_y_pt = int(box_center_y_pt - (box_height / 2))
            
            
            ############## NMS Change 2 ###############
            #save class id, start x, y, width & height, confidences in a list for nms processing
            #make sure to pass confidence as float and width and height as integers
            class_ids_list.append(predicted_class_id)
            confidences_list.append(float(prediction_confidence))
            boxes_list.append([start_x_pt, start_y_pt, int(box_width), int(box_height)])
            ############## NMS Change 2 END ###########

print('cinfidence list is',confidences_list)

############## NMS Change 3 ###############
# Applying the NMS will return only the selected max value ids while suppressing the non maximum (weak) overlapping bounding boxes      
# Non-Maxima Suppression confidence set as 0.5 & max_suppression threhold for NMS as 0.4 (adjust and try for better perfomance)
max_value_ids = cv2.dnn.NMSBoxes(boxes_list, confidences_list, 0.5, 0.4)

# loop through the final set of detections remaining after NMS and draw bounding box and write text
for max_valueid in max_value_ids:
    max_class_id = max_valueid[0]
    box = boxes_list[max_class_id]
    start_x_pt = box[0]
    start_y_pt = box[1]
    box_width = box[2]
    box_height = box[3]
    
    #get the predicted class id and label
    predicted_class_id = class_ids_list[max_class_id]
    predicted_class_label = class_labels[predicted_class_id]
    prediction_confidence = confidences_list[max_class_id]
############## NMS Change 3 END ###########

    
    #obtain the bounding box end co-oridnates
    end_x_pt = start_x_pt + box_width
    end_y_pt = start_y_pt + box_height
    
    #get a random mask color from the numpy array of colors
    box_color = class_colors[predicted_class_id]
    
    #convert the color numpy array as a list and apply to text and box
    box_color = [int(c) for c in box_color]
    
    # print the prediction in console
    predicted_class_label = "{}: {:.2f}%".format(predicted_class_label, prediction_confidence * 100)
    print("predicted object {}".format(predicted_class_label))
    
    # draw rectangle and text in the image
    cv2.rectangle(img_to_detect, (start_x_pt, start_y_pt), (end_x_pt, end_y_pt), box_color, 1)
    cv2.putText(img_to_detect, predicted_class_label, (start_x_pt, start_y_pt-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)

cinfidence list is [0.6432077884674072, 0.8346089124679565, 0.9295052886009216, 0.7703809142112732, 0.8495113253593445]
predicted object ladder: 92.95%
predicted object ladder: 84.95%


In [59]:
cv2.imshow("Detection Output", img_to_detect)

cv2.waitKey(0)

cv2.destroyAllWindows()

cv2.waitKey(1)

-1

In [9]:
path = "backend/model_config/ladder/"

In [11]:
import os

In [14]:
os.path.join(path,'image/',"dafd.img")

'backend/model_config/ladder/image/dafd.img'

In [16]:
5/3.

1.6666666666666667

In [18]:
abc = 10
def a():
    global abc
    abc=[1,2]
def b():
    print(abc)

In [19]:
a()

In [20]:
b()

[1, 2]


In [29]:
os.listdir("yolov4/darknet/images/ladders_yard/")

['0.jpg',
 '0.txt',
 '1.jpg',
 '1.txt',
 '102.jpg',
 '102.txt',
 '104.jpg',
 '104.txt',
 '106.jpg',
 '106.txt',
 '107.jpg',
 '107.txt',
 '108.jpg',
 '108.txt',
 '109.jpg',
 '109.txt',
 '110.jpg',
 '110.txt',
 '111.jpg',
 '111.txt',
 '112.jpg',
 '112.txt',
 '113.jpg',
 '113.txt',
 '114.jpg',
 '114.txt',
 '115.jpg',
 '115.txt',
 '116.jpg',
 '116.txt',
 '117.jpg',
 '117.txt',
 '118.jpg',
 '118.txt',
 '12.jpg',
 '12.txt',
 '121.jpg',
 '121.txt',
 '123.jpg',
 '123.txt',
 '124.jpg',
 '124.txt',
 '126.jpg',
 '126.txt',
 '128.jpg',
 '128.txt',
 '129.jpg',
 '129.txt',
 '13.jpg',
 '13.txt',
 '130.jpg',
 '130.txt',
 '131.jpg',
 '131.txt',
 '132.jpg',
 '132.txt',
 '133.jpg',
 '133.txt',
 '134.jpg',
 '134.txt',
 '135.jpg',
 '135.txt',
 '136.jpg',
 '136.txt',
 '137.jpg',
 '137.txt',
 '138.jpg',
 '138.txt',
 '139.jpg',
 '139.txt',
 '14.jpg',
 '14.txt',
 '140.jpg',
 '140.txt',
 '141.jpg',
 '141.txt',
 '142.jpg',
 '142.txt',
 '143.jpg',
 '143.txt',
 '144.jpg',
 '144.txt',
 '145.jpg',
 '145.txt',
 '146.

In [30]:
os.getcwd()

'D:\\WM\\Project\\ladder'

In [40]:
len(next(os.walk("."))[2])

3

In [38]:
len(f)

3

In [39]:
f

['chromedriver_win32.zip', 'debug.log', 'Untitled.ipynb']